# Bird Species Classier 
Built using the [Kaggle dataset](https://www.kaggle.com/gpiosenka/100-bird-species) by [Gerald Piosenka](https://www.kaggle.com/gpiosenka)

Project By: [Uzair](https://github.com/Uzair05)

## Download, extract and arrange data.

In [ ]:
#!pip install --upgrade kaggle
!kaggle datasets download -d gpiosenka/100-bird-species

In [ ]:
import os 
import zipfile

try:
    os.mkdir("./data/")
except Exception as err:
    print(f"Error in creating data/:\t{err}")


localzip = "./100-bird-species.zip"
zip_ref = zipfile.ZipFile(localzip, 'r')
try:
    zip_ref.extractall('data/')
except Exception as err:
    print(f"Erorr in zip extraction:\t{err}")
finally:
    zip_ref.close()

## Handle Data
Create train and test data.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

In [2]:
train_dir = "./data/train"
test_dir = "./data/test"
valid_dir = "./data/valid"

train_datagen = ImageDataGenerator(
    rescale = 1./255., 
    rotation_range = 40, 
    shear_range = 0.2,
    width_shift_range = 0.2, 
    height_shift_range = 0.2, 
    zoom_range = 0.2, 
    horizontal_flip = True, 
    vertical_flip = True, 
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    rescale=1./255.
)
valid_datagen = ImageDataGenerator(
    rescale=1./255.
)

In [3]:
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(224,224), 
    batch_size=20, 
    class_mode="categorical", 
)
test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(224,224), 
    class_mode="categorical", 
    batch_size=20
)
valid_generator = valid_datagen.flow_from_directory(
    valid_dir, 
    target_size=(224,224), 
    class_mode="categorical", 
    batch_size=20
)

Found 45980 images belonging to 315 classes.
Found 1575 images belonging to 315 classes.
Found 1575 images belonging to 315 classes.


## CNN Transfer Learning

In [3]:
from tensorflow.keras.applications.efficientnet import EfficientNetB3

In [10]:
model = tf.keras.Model()
model = tf.keras.models.load_model("./data/EfficientNetB3-birds-98.92.h5")
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 128, 128, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 128, 128, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 129, 129, 3)  0           normalization[0][0]              
_______________________________________________________________________________________

In [12]:

for layers in model.layers[:-6]:
    layers.trainable = False

In [13]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=1e-4),
              metrics=['accuracy'])

## CNN Using Own Architecture

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2), 
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2), 
    tf.keras.layers.Conv2D(32, (3,3), activation=tf.nn.leaky_relu),
    tf.keras.layers.MaxPool2D(2,2), 
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(16, activation=tf.nn.relu),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(315, activation=tf.nn.softmax)
])

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=1e-4),
              metrics=['accuracy'])

## Fit and Evaluate

In [5]:
class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs={}):
        accuracy = 0.95 # Percentage Accuracy.
        if(logs.get('accuracy') != None) and (logs.get('accuracy') >= accuracy): # Experiment with changing this value
          print(f"\nReached {accuracy*100}% accuracy so cancelling training!")
          self.model.stop_training = True

callbacks = myCallback()

In [14]:
history = model.fit(
    train_generator,
    epochs = 5, 
    #steps_per_epoch = 50, 
    validation_data = test_generator, 
    verbose = 1, 
    validation_steps = 10,
    callbacks=[callbacks]
)

NameError: name 'train_generator' is not defined

In [ ]:
tf.keras.models.save_model(model, "./data", save_format='h5')

In [ ]:
history.history

In [ ]:
eval = model.evaluate(
    valid_generator,
    batch_size=20
)